In [14]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils.np_utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import os
import cv2
%matplotlib inline
import pickle
import tensorflow as tf


In [15]:
TRAIN_DATA_PATH = r''
NUM_OF_LETTERS = 26 # "A" to "Z" 
IMAGE_SIZE = 50 #We'll be workign with 50 * 50 pixel images
NUM_OF_CHANNELS = 1 # Grayscale
NUM_OF_DENSE_LAYER_NODES = (IMAGE_SIZE * IMAGE_SIZE) // 2

LABELS = ['a','b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
        'n','o', 'p', 'q','r', 's', 't','u', 'v', 'w', 'x', 'y', 'z']
print(len(LABELS))

26


In [16]:
y_unique = []
x_unique = []
for alphabet in os.listdir(TRAIN_DATA_PATH):
    for image in os.listdir(TRAIN_DATA_PATH+str('\\')+alphabet):
        X=imread(TRAIN_DATA_PATH+str('\\')+alphabet+str('\\')+image)
        x_unique.append(X)
        y_unique.append(alphabet)
        break

In [1]:
fig=plt.figure(figsize=(12,12))
for i in range(len(x_unique)):
    ax=fig.add_subplot(6,6,i+1)
    ax.imshow(x_unique[i])
    plt.xlabel(y_unique[i])
    plt.xticks([])
    plt.yticks([])
plt.axis('off')
plt.show()
fig.savefig('alphabet')

NameError: name 'plt' is not defined

In [3]:
def create_data(DATA_PATH):
    x=[]
    y=[]
    paths=[]
    for label in LABELS:
        path = os.path.join(DATA_PATH, label)
        label_name = LABELS.index(label)
        for img in os.listdir(path):
            p=os.path.join(path, img)
            paths.append(p)
            print(p)
            try:
                img_array = cv2.imread(p)#os.path.join(path, img))
                img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
                img_array = cv2.resize(img_array, (IMAGE_SIZE, IMAGE_SIZE))
                x.append(img_array)
                y.append(label_name)
            except Exception as e:
                pass
    return x,y

X_train, y_train= create_data(TRAIN_DATA_PATH)


NameError: name 'TRAIN_DATA_PATH' is not defined

In [19]:
NUM_OF_TRAIN_IMAGES = len(X_train)
print(len(X_train))

18150


In [20]:
X_train=np.array(X_train)

In [21]:
X_train = X_train.reshape(NUM_OF_TRAIN_IMAGES, IMAGE_SIZE , IMAGE_SIZE, NUM_OF_CHANNELS)
print(X_train.shape)

(18150, 50, 50, 1)


In [22]:
X_train = X_train.astype("float32")
X_train /= 255.0


In [23]:
y_train = to_categorical(y_train, NUM_OF_LETTERS)
print(X_train.shape)


(18150, 50, 50, 1)


In [24]:
#Saving Data
np.save("X_train.npy", X_train)
np.save("y_train.npy", y_train)

In [25]:
X_train=np.load("X_train.npy")
y_train=np.load("y_train.npy")

# Creating CNN Model

In [26]:
model = Sequential()

model.add(Conv2D(IMAGE_SIZE, (3, 3), padding = "same", input_shape = (IMAGE_SIZE, IMAGE_SIZE, NUM_OF_CHANNELS), activation = "relu"))
model.add(Conv2D(IMAGE_SIZE, (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(2 * IMAGE_SIZE, (3, 3), padding = "same", activation = "relu"))
model.add(Conv2D(2 * IMAGE_SIZE, (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(NUM_OF_DENSE_LAYER_NODES, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(NUM_OF_LETTERS, activation = (tf.nn.softmax)))



# Compiling CNN Model

In [27]:
model.compile(
    optimizer = "adam", 
    loss = "categorical_crossentropy", 
    metrics = ["accuracy"]
)

In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 50, 50, 50)        500       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 48, 48, 50)        22550     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 24, 24, 50)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 24, 24, 50)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 24, 24, 100)       45100     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 22, 22, 100)       90100     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 11, 11, 100)       0         
__________

# Training Model

In [29]:
history = model.fit(
    X_train, 
    y_train, 
    batch_size = 52,
    epochs = 5,
    shuffle = True
)

Epoch 1/5
18150/18150 [==============================] - 1083s 60ms/step - loss: 0.6636 - acc: 0.7835
Epoch 2/5
18150/18150 [==============================] - 1044s 58ms/step - loss: 0.1090 - acc: 0.9624
Epoch 3/5
18150/18150 [==============================] - 1092s 60ms/step - loss: 0.0540 - acc: 0.9822
Epoch 4/5
18150/18150 [==============================] - 916s 50ms/step - loss: 0.0512 - acc: 0.9829
Epoch 5/5
18150/18150 [==============================] - 594s 33ms/step - loss: 0.0370 - acc: 0.9881


# Saving Model for future use

In [30]:
#Saving Model
from keras.models import load_model
model.save('model.h5') 